In [1]:
# Imports
import torch
import numpy as np
import os
import torchvision
import json
import PIL
import matplotlib.pyplot as plt
import copy
from collections import OrderedDict
import torch.nn.functional as F
import torchvision.transforms as tr
import torch.nn as nn

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

gpu = torch.cuda.is_available()
gpu

True

In [2]:
# Sets
phases = ['train', 'valid', 'test']

In [3]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [4]:
# Transforms
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transforms = {}

# transforms1 for training set
# transforms[phases[0]] = tr.Compose([
#     tr.RandomRotation((0, 45)),
#     tr.Resize(224),
#     tr.CenterCrop(224),
#     tr.RandomApply([tr.Pad((96, 0, 0, 0))], p=0.25),
#     tr.RandomApply([tr.Pad((0, 96, 0, 0))], p=0.25),
#     tr.RandomApply([tr.Pad((0, 0, 0, 96))], p=0.25),
#     tr.CenterCrop(224),
#     tr.RandomHorizontalFlip(),
#     tr.ToTensor(),
#     tr.Normalize(mean, std)
# ])

# transforms2 for training set
transforms[phases[0]] = tr.Compose([
    tr.RandomRotation((0, 45)),
    tr.Resize(224),
    tr.CenterCrop(224),
    tr.RandomApply([tr.Pad((48, 0, 0, 0))], p=0.5),
    tr.RandomApply([tr.Pad((0, 48, 0, 0))], p=0.5),
    tr.RandomApply([tr.Pad((0, 0, 0, 48))], p=0.5),
    tr.CenterCrop(224),
    tr.RandomHorizontalFlip(),
    tr.ToTensor(),
    tr.Normalize(mean, std)
])

transforms[phases[1]] = tr.Compose([
#     tr.RandomRotation((0, 45)),
    tr.Resize(224),
    tr.CenterCrop(224),
    tr.RandomHorizontalFlip(),
    tr.ToTensor(),
    tr.Normalize(mean, std)
])

# Transforms for testing set
transforms[phases[2]] = tr.Compose([
#     tr.RandomRotation((0, 45)),
    tr.Resize(224),
    tr.CenterCrop(224),
    tr.ToTensor(),
    tr.Normalize(mean, std)
])

In [5]:
dir_path = 'flower_data'
dirs = {}
shuffle_dataloader = {phases[0]: True, phases[1]: False, phases[2]: False}
datasets = {}
dataloaders = {}
batch_size = 8
for phase in phases:
    dirs[phase] = dir_path + '/' + phase
    datasets[phase] = torchvision.datasets.ImageFolder(
        dirs[phase], 
        transform=transforms[phase]
    )
    
    dataloaders[phase] = torch.utils.data.DataLoader(
        datasets[phase], 
        batch_size=batch_size, 
        shuffle=shuffle_dataloader[phase]
    )

In [6]:
# fig, ax = plt.subplots()
# images, labels = next(iter(dataloaders['test']))
# ax.imshow(images[0].numpy().transpose((1, 2, 0)))
# images[0].size()

In [7]:
model = torchvision.models.resnet152(pretrained=True)
for param in model.parameters():
    param.requires_grad = True
# First iteration of the classifier
# classifier1 = nn.Sequential(OrderedDict([
#     ('fc1', nn.Linear(2048, 1024, bias=False)),
#     ('relu1', nn.ReLU()),
#     ('dropout1', nn.Dropout(p=0.25, inplace=True)),
#     ('fc2', nn.Linear(1024, 256, bias=False)),
#     ('relu2', nn.ReLU()),
#     ('dropout2', nn.Dropout(p=0.25, inplace=True)),
#     ('fc3', nn.Linear(256, 102, bias=False))
# ]))
# model.fc = classifier1

# Testing Bias = True and dropout = 0.5
# classifier2 = nn.Sequential(OrderedDict([
#     ('fc1', nn.Linear(2048, 1024, bias=False)),
#     ('relu1', nn.ReLU()),
#     ('dropout1', nn.Dropout(p=0.5, inplace=True)),
#     ('fc2', nn.Linear(1024, 102, bias=True))
# ]))
# model.fc = classifier2

# classifier3 = nn.Sequential(OrderedDict([
#     ('fc1', nn.Linear(2048, 102, bias=True))
# ]))
# model.fc = classifier3

classifier4 = nn.Sequential(OrderedDict([
    ('dropout1', nn.Dropout(p=0.1)),
    ('relu1', nn.ReLU()),
    ('fc1', nn.Linear(2048, 102, bias=True))
]))
model.fc = classifier4

print(model.fc)
if gpu:
    model.cuda()

Sequential(
  (dropout1): Dropout(p=0.1)
  (relu1): ReLU()
  (fc1): Linear(in_features=2048, out_features=102, bias=True)
)


In [13]:
def save_valid_loss_min(filepath, valid_loss_min):
    valid_loss_min=valid_loss_min
    torch.save(valid_loss_min, filepath)
    
def load_valid_loss_min(filepath):
    valid_loss_min = torch.load(filepath)
    return valid_loss_min

def save_final_checkpoint(filepath):
    checkpoint = {
        'state_dict': model.state_dict(),
        'classifier': model.fc
                 }
    torch.save(checkpoint, filepath)
    
def load_final_checkpoint(checkpoint):
    checkpoint = torch.load(checkpoint)
    model = torchvision.models.resnet152(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    
    model.fc = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    if torch.cuda.is_available():
        model.cuda()
    return model

def save_cp(filepath):
    checkpoint = {
        'state_dict': model.state_dict()
    }
    torch.save(checkpoint, filepath)

In [16]:
save_cp('pytorch0.4_cp.pth')

In [24]:
# Defining criterion and optimizer
criterion = nn.CrossEntropyLoss()
valid_loss_min = np.Inf
best_acc = 0.0
start_lr = 0.01
lr = start_lr

In [15]:
valid_loss_min = load_valid_loss_min('valid_loss_min.pth')
model = load_final_checkpoint('final_checkpoint.pth')
print(valid_loss_min)

0.0842045000537975


In [29]:
# number of epochs to train the model
n_epochs = 50
# About how many epochs can go without improving loss
max_epoch_reset = 10
perc = 1.1
lr = 0.00001
epoch_save = 0
mstep1 = int(n_epochs * 0.25)
mstep2 = int(n_epochs * 0.5)
mstep3 = int(n_epochs * 0.75)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [mstep1, mstep2, mstep3], 0.5)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

In [30]:
# TEST ONLY
phases = ['test']

In [31]:
# TRAINING AND TESTING
phases = ['train', 'valid', 'test']

In [32]:
for epoch in range(1, n_epochs+1):
    steps_lr = lr
    if phases != ['test']:
        print('-' * 50)
        print(f'Epoch: {epoch}')
    scheduler.step()
    for phase in phases:
        steps = 0
        phase_loss = 0.0
        phase_running_corrects = 0
        
        if phase == 'test':
            if epoch < n_epochs: break
            print(f'>>> Loading previous best model with Validation Loss of {valid_loss_min}')
            valid_loss_min = load_valid_loss_min('valid_loss_min.pth')
            model = load_final_checkpoint('final_checkpoint.pth')
        for data, target in dataloaders[phase]:
            steps += 1
            if gpu:
                data, target = data.cuda(), target.cuda()
            if phase == 'train':
                model.train()
#                 steps_lr = lr * 0.98
#                 optimizer = torch.optim.SGD(
#                     model.parameters(), 
#                     lr=steps_lr
#                 )
                optimizer.zero_grad()
                output = model.forward(data)
                loss = criterion(output, target)
            else:
                model.eval()
                with torch.no_grad():
                    output = model.forward(data)
                    loss = criterion(output, target)                     

            top_p, top_class = output.topk(1)
            equals = top_class == target.view(*top_class.shape)
            # calculate the batch loss
            
            # backward pass: compute gradient of the loss with respect to model parameters
            if phase == 'train':
                loss.backward()
                optimizer.step()
            # update training loss
            phase_loss += loss.item()*data.size(0)
            phase_running_corrects += torch.sum(equals)

        # Training Loss and Accuracy
        phase_loss = phase_loss / len(datasets[phase])
        phase_epoch_acc = phase_running_corrects.double() / len(datasets[phase])
        print(f'Phase: {phase}\tLoss: {phase_loss:.6f} \tAccuracy: {phase_epoch_acc:.4f}')
        if epoch == 1: print(f'Steps: {steps}')
        # Save model if validation loss has decreased
        delta_epochs = epoch - epoch_save
        if phase == 'valid':
            epoch_reset = min(max(int(round(perc * n_epochs)), 2), max_epoch_reset)
            if phase_loss <= valid_loss_min:
                print(f'Validation loss decreased ({valid_loss_min:.4f} to {phase_loss:.4f}). Model saved.')
                epoch_save = epoch
                valid_loss_min = phase_loss
                best_acc = phase_epoch_acc
#                 save_checkpoint('checkpoint.pth', valid_loss_min, lr)
                save_valid_loss_min('valid_loss_min.pth', valid_loss_min)
                save_final_checkpoint('final_checkpoint.pth')

            elif (valid_loss_min != np.Inf and delta_epochs == epoch_reset):
                epoch_save = epoch
                valid_loss_min = load_valid_loss_min('valid_loss_min.pth')
                model = load_final_checkpoint('final_checkpoint.pth')
                print(f'>>> Loading previous best model with Validation Loss of {valid_loss_min}')
        

--------------------------------------------------
Epoch: 1
Phase: train	Loss: 0.092006 	Accuracy: 0.9782
Steps: 819
Phase: valid	Loss: 0.091410 	Accuracy: 0.9756
Steps: 103
--------------------------------------------------
Epoch: 2
Phase: train	Loss: 0.085710 	Accuracy: 0.9834
Phase: valid	Loss: 0.104181 	Accuracy: 0.9780
--------------------------------------------------
Epoch: 3
Phase: train	Loss: 0.086010 	Accuracy: 0.9820
Phase: valid	Loss: 0.096317 	Accuracy: 0.9780
--------------------------------------------------
Epoch: 4
Phase: train	Loss: 0.083938 	Accuracy: 0.9808
Phase: valid	Loss: 0.104753 	Accuracy: 0.9756
--------------------------------------------------
Epoch: 5
Phase: train	Loss: 0.083498 	Accuracy: 0.9823
Phase: valid	Loss: 0.093445 	Accuracy: 0.9780
--------------------------------------------------
Epoch: 6
Phase: train	Loss: 0.092092 	Accuracy: 0.9785
Phase: valid	Loss: 0.100760 	Accuracy: 0.9792
--------------------------------------------------
Epoch: 7
Phase:

In [37]:
print(f'Phase: {phase}\tLoss: {phase_loss:.6f} \tAccuracy: {phase_epoch_acc:.4f}')

Phase: test	Loss: 0.094320 	Accuracy: 0.9768


In [ ]:
# helper function for visualizing the output of a given layer
# default number of filters is 4
def viz_layer(layer, n_filters= 4):
    fig = plt.figure(figsize=(n_filters, 50))
    
    for i in range(n_filters):
        ax = fig.add_subplot(n_filters, 8, i+1, xticks=[], yticks=[])
        # grab layer outputs
        ax.imshow(np.squeeze(layer[0,i].data.numpy()), cmap='gray')
        ax.set_title('Output %s' % str(i+1))

In [ ]:
model.eval()
model.cpu()
data, target = next(iter(dataloaders['train']))
with torch.no_grad():
    _, pool1, pool2, pool3, pool4 = model.forward(data)
viz_layer(pool1, n_filters=16)

model.cuda()

In [ ]:
viz_layer(pool2, n_filters=16)

In [ ]:
viz_layer(pool3, n_filters=16)

In [ ]:
valid_epoch_acc